In [ ]:
from langchain.llms import HuggingFaceTextGenInference, OpenAI
from dotenv import load_dotenv
load_dotenv()

Wir definieren nun zwei Hilfsfunktionen um einfach zwischen GPT4 und unserem lokalen Sprachmodell zu wechseln.

In [50]:
def getLLM_openai():
    return OpenAI(request_timeout=10) #10s timeout, model_name = "gpt-3.5-turbo"


def getLLM_TGI(): 
     inference_server_url_local = "" 
     llm = HuggingFaceTextGenInference(inference_server_url=inference_server_url_local, max_new_tokens=2048, top_k=10, top_p=0.95, typical_p=0.95, temperature=0.01, repetition_penalty=1.03) 
     return llm
 
 
llm = getLLM_TGI()

# Erste Experimente
In der nächsten Zelle rufen wir das LLM einfach auf.

In [51]:
res = llm.invoke("Warum ist der Himmel blau?")
print(res)



Der Himmel erscheint blau, weil die Luft Moleküle enthält, die Licht in verschiedenen Wellenlängen absorbieren. Wenn das Sonnenlicht in den Weltraum gelangt, wird es von der Atmosphäre der Erde aufgesaugt. Einige dieser Lichtwellen werden von den Molekülen in der Atmosphäre absorbiert und andere werden reflektiert.

Die Moleküle in der Atmosphäre absorbieren Licht in verschiedenen Wellenlängen, aber sie absorbieren Licht in der blauen Wellenlänge am besten. Das bedeutet, dass die Moleküle in der Atmosphäre mehr blaues Licht absorbieren als jede andere Farbe.

Das, was uns als blau erscheint, ist das Licht, das nicht von den Molekülen in der Atmosphäre absorbiert wird. Das Licht, das nicht absorbiert wird, wird reflektiert und trifft unsere Augen. Daher erscheint der Himmel blau, weil die Moleküle in der Atmosphäre das blaue Licht absorbieren und das restliche Licht reflektieren.

Warum ist der Himmel nicht immer blau?

Obwohl der Himmel normalerweise blau erscheint, kann er manchmal 

# RAG workflow
Im folgenden stellen wir einen Beispielhaften RAG Workflow dar, der Daten von der Wikipedia Seite von Tibet als Grundlage nimmt.

In [52]:
import prompts
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, GPT4AllEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

loader = WebBaseLoader("https://de.wikipedia.org/wiki/Tibet")
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
splits = text_splitter.split_documents(loader.load())

Wir definieren uns nun ein Embedding für die deutsche Sprache und nehmen Chroma als Vektordatenbank. Diese benötigen wir für die Suche nach passenden Dokumenten für unsere Anfrage an das Sprachmodell

In [53]:
emb = HuggingFaceEmbeddings(model_name='aari1995/German_Semantic_STS_V2')
vectorstore = Chroma.from_documents(documents=splits,embedding=emb, persist_directory="./chroma_db")
retriever = vectorstore.as_retriever()
rag_prompt = prompts.german_sauerkrautlm()

No sentence-transformers model found with name aari1995/German_Semantic_STS_V2. Creating a new one with MEAN pooling.


In der folgenden Zelle verketten wir die Anfrage an die Vektordatenbank mit dem Aufruf von unserem Sprachmodell in einem `rag_chain`.

In [54]:
# configure rag chain
chain_type_kwargs = {"prompt": rag_prompt}
rag_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, 
                                    chain_type_kwargs=chain_type_kwargs,
                                    return_source_documents=True,
                                    )

Nun erfolgt die eigentliche Anfrage an das LLM:

In [59]:
res = rag_chain.invoke("Was weißt du zu Tibet?")
print(res['result'])

 Tibet ist eine Region in Zentralasien, die den Lebensraum tibetisch-sprachiger Völker umfasst. Es ist bekannt für seine eigene Kultur und seine eigenständigen Staaten, die sich über Teile des tibetischen Hochlands erstreckten. Tibet besaß bis ins 20. Jahrhundert hinein ein eigenes Staatswesen, aber seine gegenwärtige Zugehörigkeit zur Volksrepublik China ist völkerrechtlich umstritten. Es gibt eine tibetische Exilregierung, die von vielen Ländern unterstützt wird. Tibet ist auch für seinen Buddhismus bekannt, insbesondere für den Vajrayana-Buddhismus. Der 14. Dalai Lama ist ein bedeutender Vertreter dieser Schule. Die vorbuddhistische tibetische Religion ist der Bön.

Quelle(n):
Lage von Tibet auf einer Weltkarte
Yangbajain-Tal, etwa 100 Kilometer nordwestlich von Lhasa
Fährverkehr über den Brahmaputra (Yarlung Zangbo)
Tibet ist eine Region in Zentralasien, die den Lebensraum tibetisch-sprachiger Völker umfasst. Abgeschieden durch das Himalaya-Gebirge am Südrand hat Tibet eine eigenst